In [22]:
class DASH:
    def __init__(self):
        
        ## layout
        self.grid_ = []
        self.tab_ = None
        
        ## Panel controls
        self.main_panel_ = []
        self.aux_panel_ = []
        self.tab_titles_ = []
        
        # self.output_ = widgets.Output()
        
    def run(self):
        return self.grid_

    def gui(self, **kwargs):
        for key in kwargs.keys():
            setattr(self, key, kwargs[key])

    def calculate(self, button):
        pass

    def update(self, button):
        pass
    
    def create_grid(self):

        ## Grid size
        self.grid_ = GridspecLayout(max(14, len(self.main_panel_)+len(self.aux_panel_)), 6, height="720px")

        ## Buttons
        calculate_button = widgets.Button(
            description="Calculate",
            layout=Layout(width="91%", border="2px solid gray"),
        )
        calculate_button.on_click(self.calculate)
        
        update_button = widgets.Button(
            description="Update Visualization",
            layout=Layout(width="91%", border="2px solid gray"),
        )
        update_button.on_click(self.update)
        
        ## Left panel layout
        panel_layout = [calculate_button]
        panel_layout += [
            widgets.HBox(
                [
                    widgets.Label(value=self.main_panel_[index]["desc"]),
                    self.main_panel_[index]["widget"],
                ],
                layout=Layout(
                    display="flex",
                    justify_content="flex-end",
                    align_content="center",
                ),
            )
            for index in range(len(self.main_panel_))
        ]
        panel_layout += [update_button]
        panel_layout += [
            widgets.HBox(
                [
                    widgets.Label(value=self.aux_panel_[index]["desc"]),
                    self.aux_panel_[index]["widget"],
                ],
                layout=Layout(
                    display="flex",
                    justify_content="flex-end",
                    align_content="center",
                ),
            )
            for index in range(len(self.aux_panel_))
        ]

        self.grid_[:, 0] = widgets.VBox(panel_layout)
            

        ## Output tabs
        self.output_ = [widgets.Output() for _ in range(len(self.tab_titles_))]
        self.tab_ = widgets.Tab()
        self.tab_.children = [
            widgets.VBox(
                [self.output_[i]], layout=Layout(height="650px", border="2px solid gray")
            )
            for i, _ in enumerate(self.tab_titles_)
        ]
        for i, title in enumerate(self.tab_titles_):
            self.tab_.set_title(i, title)
        self.grid_[:, 1:] = self.tab_

        ## interactive
        args = {
            **{control["arg"]: control["widget"] for control in self.main_panel_},
            **{control["arg"]: control["widget"] for control in self.aux_panel_},
        }
        # with self.output_:
        display(widgets.interactive_output(self.gui, args,))
            
            
import pandas as pd
filepath = "../data/papers/urban-agriculture-CL.csv"
df = pd.read_csv(filepath)

import ipywidgets as widgets
from ipywidgets import AppLayout, GridspecLayout, Layout
from techminer.params import EXCLUDE_COLS
import techminer.gui as gui
            
class DASHapp(DASH):
    def __init__(self, data, limit_to, exclude):
        
        super(DASH, self).__init__()
        
        self.data_ = data
        self.limit_to_ = limit_to
        self.exclude_ = exclude
        self.tab_titles_ = ['1', '2', '3']

        COLUMNS = sorted(
            [column for column in data.columns if column not in EXCLUDE_COLS]
        )
        
        self.main_panel_ = [
            gui.dropdown(
                desc="View:",
                options=["Matrix", "Heatmap", "Bubble plot", "Network", "Slope chart",],
            ),
            gui.dropdown(
                desc="Column:", options=[z for z in COLUMNS if z in data.columns],
            ),
            gui.dropdown(
                desc="By:", options=[z for z in COLUMNS if z in data.columns],
            ),
            gui.dropdown(
                desc="Top by:", options=["Num Documents", "Times Cited", "Data",],
            ),
            gui.top_n(),
        ]
        
        self.aux_panel_ = [
            gui.dropdown(
                desc="Sort C-axis by:",
                options=["Alphabetic", "Num Documents", "Times Cited", "Data",],
            ),
            gui.c_axis_ascending(),
            gui.dropdown(
                desc="Sort R-axis by:",
                options=["Alphabetic", "Num Documents", "Times Cited", "Data",],
            ),
            gui.r_axis_ascending(),
            gui.cmap(arg="cmap", desc="Colormap Col:"),
            gui.cmap(arg="cmap_by", desc="Colormap By:"),
            gui.nx_layout(),
            gui.nx_max_iter(),
            gui.fig_width(),
            gui.fig_height(),            
        ]
        super().create_grid()
        
    def gui(self, **kwargs):

        super().gui(**kwargs)        
        
    def calculate(self, button):
        with self.output_[self.tab_.selected_index]:
            display('calculate in tab '+ str(self.tab_.selected_index))
        

    def update(self, button):
        with self.output_[self.tab_.selected_index]:
            display('updata in tab '+ str(self.tab_.selected_index))
    
    
DASHapp(data=df, limit_to=None, exclude=None).run()

Output()

GridspecLayout(children=(VBox(children=(Button(description='Calculate', layout=Layout(border='2px solid gray',…